# Case Study: Introduction to Python Programming

This case study demonstrates various concepts learned in the first module, such as loops, functions, conditional statements, and lambda functions. The stock-level data of five major U.S. companies is fetched within the program and used to illustrate these concepts in practical examples.


In [1]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Let us obtain data od one stock for the financial year 2024. Note that the column names have two levels 
#and we want to use first level 

data = yf.download('META',start="2023-01-01", end="2024-01-01", progress=False)
data.head()

YF.download() has changed argument auto_adjust default to True



1 Failed download:
['META']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,META,META,META,META,META,META
Date,,,,,,


In [3]:
# Now We create a function named 'download()' which accepts ticker (stock name), start date and end date as parameters
#The function returns open, high, low, close and volume for the ticker
#We select first level of column names
def download(ticker, start="2023-01-01", end="2024-01-01"):
    # Download data
    data = yf.download(ticker, start=start, end=end, progress=False)
    data.columns = [col[0] for col in data.columns.values] 
    # Reset index to convert date index to column
    data = data.reset_index()
    return data


In [4]:
#Let us create list of stocks for which data is required
#FAANG - 'META','AMAZON','APPLE','NETFLIX','GOOGLE'

faang_list = ['META','AAPL','AMZN','NFLX','GOOGL']

In [16]:
#We will form a loop and apply download() on each of these tickers
def get_data(stock_list, local: bool = False):
    result_dfs = []
    if local:
        for ticker in stock_list:
            df = pd.read_csv(f"{ticker}.csv")
            result_dfs.append(df)
    else:
        for ticker in stock_list:
            df = download(ticker)
            if df.empty:
                continue
            df['Ticker'] = ticker  # optional: keep track of stock name
            df.to_csv(df['Ticker'] + '.csv', index=False)
            result_dfs.append(df)
    return result_dfs

faang_dfs = get_data(faang_list)


1 Failed download:
['META']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['AMZN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['NFLX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['GOOGL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [6]:
faang_dfs

[Empty DataFrame
 Columns: [Date, Adj Close, Close, High, Low, Open, Volume, Ticker]
 Index: [],
 Empty DataFrame
 Columns: [Date, Adj Close, Close, High, Low, Open, Volume, Ticker]
 Index: [],
 Empty DataFrame
 Columns: [Date, Adj Close, Close, High, Low, Open, Volume, Ticker]
 Index: [],
 Empty DataFrame
 Columns: [Date, Adj Close, Close, High, Low, Open, Volume, Ticker]
 Index: [],
 Empty DataFrame
 Columns: [Date, Adj Close, Close, High, Low, Open, Volume, Ticker]
 Index: []]

In [7]:
faang_dfs[0] #Meta

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker


In [8]:
#We create a function named 'calculate_return' which takes list object as an argument
#Calculates return using variable 'Close'
#Further, it derives the 'Direction' as 1 and 0 using the concept of lamda function

def calculate_return(df):
    df['Prev_Close'] = df['Close'].shift(1)
    try:
        df['Returns'] = ((df['Close'] - df['Prev_Close']) / df['Prev_Close']) * 100
    except Exception as e:
        print("⚠️ Error calculating returns:", e)
    df['Returns'] = df['Returns'].fillna(0)
    
    df['Direction'] = df.apply(lambda x: 1 if x['Returns'] > 0 else 0, axis=1)
    df.drop(columns='Prev_Close', inplace=True)
    return df


In [9]:
#Now we apply the function calculate_returns() for each object in a list
processed_dfs = []
for df in faang_dfs:
    processed_df = calculate_return(df)
    processed_dfs.append(processed_df)

faang_dfs = processed_dfs

In [10]:
faang_dfs[0]

,Date,Adj Close,Close,High,Low,Open,Volume,Ticker,Returns,Direction


In [11]:
#We create a function 'monthly_returns' which derives month from the 'Date' column and calculates return for each month

def monthly_returns(df):
    ticker = df['Ticker']
    # Ensure Date column is datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # Derive Year-Month
    df['Month'] = df['Date'].dt.to_period('M')
    
    # Group by Month and calculate monthly return
    monthly_df = (
        df.groupby('Month')
        .apply(lambda x: ((x['Close'].iloc[-1] - x['Close'].iloc[0]) / x['Close'].iloc[0]) * 100)
        .reset_index(name='Monthly_Return_%')
    )
    monthly_df['Ticker'] = ticker

    return monthly_df.round(3)


In [12]:
# Now we apply the function monthly_returns() for each object in a list
monthly_dfs = [] #creates empty list
for df in faang_dfs:
    monthly_df = monthly_returns(df)
    monthly_dfs.append(monthly_df)

monthly_dfs[1] #Accessing second item in a list

TypeError: DataFrame.reset_index() got an unexpected keyword argument 'name'

In [ ]:
monthly_dfs #Displays all list items